# Choropleth Map

Display the distribution of dengue cases across different states or regions in India. This visualizes the geographic spread and intensity of outbreaks.

### Inspect: 

Run the inspection code to find the correct column name for state identifiers.

In [8]:
import geopandas as gpd

# Load the geospatial data for Indian states
geo_data_url = 'https://raw.githubusercontent.com/geohacker/india/master/state/india_telengana.geojson'
geo_data = gpd.read_file(geo_data_url)

# Inspect the GeoDataFrame
print("Columns in GeoDataFrame:", geo_data.columns)
print("First few rows of the GeoDataFrame:")
print(geo_data)

Columns in GeoDataFrame: Index(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'NAME_1', 'geometry'], dtype='object')
First few rows of the GeoDataFrame:
    ID_0  ISO NAME_0  ID_1                  NAME_1  \
0    105  IND  India     1     Andaman and Nicobar   
1    105  IND  India    36               Telangana   
2    105  IND  India     2          Andhra Pradesh   
3    105  IND  India     3       Arunachal Pradesh   
4    105  IND  India     4                   Assam   
5    105  IND  India     5                   Bihar   
6    105  IND  India     6              Chandigarh   
7    105  IND  India     7            Chhattisgarh   
8    105  IND  India     8  Dadra and Nagar Haveli   
9    105  IND  India     9           Daman and Diu   
10   105  IND  India    10                   Delhi   
11   105  IND  India    11                     Goa   
12   105  IND  India    12                 Gujarat   
13   105  IND  India    13                 Haryana   
14   105  IND  India    14        Himachal Pradesh

### Generate the Choropleth Map

In [13]:
import geopandas as gpd
import pandas as pd
import folium
from IPython.display import display, IFrame

# Load the geospatial data for Indian states
geo_data_url = 'https://raw.githubusercontent.com/geohacker/india/master/state/india_telengana.geojson'
geo_data = gpd.read_file(geo_data_url)

# Print the columns and a few rows to confirm the correct state name column
'''
print("Columns in GeoDataFrame:", geo_data.columns)
print("First few rows of the GeoDataFrame:")
print(geo_data.head())
'''

# Confirmed state name column is 'NAME_1'
state_name_column = 'NAME_1'

# Sample data for dengue cases per state (ensure these state names match those in geo_data)
dengue_cases = {
    'State': ['Uttar Pradesh', 'Kerala', 'Karnataka', 'Maharashtra', 'Bihar', 'West Bengal', 'Tamil Nadu', 'Rajasthan'],
    'Cases': [5000, 15000, 11000, 12000, 9000, 14000, 8000, 7000]
}

# Create a DataFrame from the dengue cases data
df_cases = pd.DataFrame(dengue_cases)

# Standardize the state names to ensure matching
df_cases['State'] = df_cases['State'].str.strip().str.title()

# Get the unique state names from both datasets for comparison
geo_state_names = geo_data[state_name_column].str.strip().str.title().unique()
dengue_state_names = df_cases['State'].unique()

# Find mismatches between the two sets of state names
mismatched_states = set(dengue_state_names) - set(geo_state_names)
missing_states_in_data = set(geo_state_names) - set(dengue_state_names)

# Print mismatches if any
'''
if mismatched_states:
    print("States in dengue data but not in GeoDataFrame:", mismatched_states)
else:
    print("All state names in dengue data match those in the GeoDataFrame.")

if missing_states_in_data:
    print("States in GeoDataFrame but not in dengue data:", missing_states_in_data)
else:
    print("All states in the GeoDataFrame are represented in the dengue data.")

'''
# Merge the geospatial data with the dengue cases data, keeping 'NAME_1' as a column
merged_data = geo_data.set_index(state_name_column).join(df_cases.set_index('State')).reset_index()

# Filter merged_data to include only rows with data in 'Cases' column
merged_data = merged_data.dropna(subset=['Cases'])

# Check if the merge was successful
'''
print("Merged DataFrame head:")
print(merged_data.head())
'''

# Create a base map centered around India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5, tiles='cartodb positron')

# Add the choropleth map
folium.Choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=merged_data,
    columns=[state_name_column, 'Cases'],
    key_on=f'feature.properties.{state_name_column}',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Dengue Cases',
    nan_fill_color='white'  # Fill color for states without data
).add_to(m)

# Add state borders for better clarity
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    }
).add_to(m)

# Save the map to an HTML file
map_file = 'dengue_cases_choropleth_map.html'
m.save(map_file)

print(f"Choropleth map saved as {map_file}")


# Display the map in the notebook
display(IFrame(map_file, width=900, height=800))



Choropleth map saved as dengue_cases_choropleth_map.html
